In [1]:
from BackgammonModel import *

In [2]:
model = BackgammonModel( \
    input_size = 198,
    hidden_size = 50,
    output_size = 1,
    trace_decay = 0.7,
    alpha = 0.1,
    save_interval = 1000,
    saved_model = './checkpoints/50/checkpoint_358000.pt',
    eval_interval = 100
)

Loading saved model: ./checkpoints/50/checkpoint_358000.pt


In [3]:
model2 = BackgammonModel( \
    input_size = 198,
    hidden_size = 50,
    output_size = 1,
    trace_decay = 0.7,
    alpha = 0.1,
    save_interval = 1000,
    saved_model = './checkpoints/50/checkpoint_333000.pt',
    eval_interval = 1000
)

Loading saved model: ./checkpoints/50/checkpoint_333000.pt


In [4]:
env = Env()
model.evaluate(env, [RandomAgent(player = 0), 
                     RandomAgent(player = 1)])

100%|██████████| 100/100 [00:07<00:00, 13.14it/s]


{0: 56, 1: 44, <Player.EMPTY: 2>: 0}

In [5]:
env = Env()
model.evaluate(env, [TDAgent(player = 0, net = model2, env = env),
                     TDAgent(player = 1, net = model, env = env)])

100%|██████████| 100/100 [00:23<00:00,  4.34it/s]


{0: 46, 1: 50, <Player.EMPTY: 2>: 4}

In [6]:
env = Env()
model.evaluate(env, [TDAgent(player = 0, net = model, env = env),
                     TDAgent(player = 1, net = model2, env = env)])

100%|██████████| 100/100 [00:23<00:00,  4.24it/s]


{0: 53, 1: 45, <Player.EMPTY: 2>: 2}

In [7]:
env = Env()
model.evaluate(env, [RandomAgent(player = 0),
                     TDAgent(player = 1, net = model2, env = env)])

100%|██████████| 100/100 [00:12<00:00,  8.20it/s]


{0: 1, 1: 99, <Player.EMPTY: 2>: 0}

In [8]:
model.evaluate(env, [TDAgent(player = 0, net = model2, env = env),
                     RandomAgent(player = 1),
                     ])

100%|██████████| 100/100 [00:12<00:00,  8.21it/s]


{0: 99, 1: 1, <Player.EMPTY: 2>: 0}

In [9]:
#model.train(358001,200000)

In [10]:
from BackgammonModel import *

In [11]:
%load_ext line_profiler
model = BackgammonModel( \
    input_size = 198,
    hidden_size = 50,
    output_size = 1,
    trace_decay = 0.7,
    alpha = 0.1,
    save_interval = 1000,
    saved_model = './checkpoints/50/checkpoint_358000.pt',
    eval_interval = 100
)
env = Env()
agents = [RandomAgent(player = 0), TDAgent(player = 1, net = model, env = env)]

Loading saved model: ./checkpoints/50/checkpoint_358000.pt


In [12]:
p, _, _, _ = env.reset()
agent = agents[p]
legal_actions = env.get_legal_actions()
print(legal_actions)
game = env.game

[0, 416, 468, 676, 962, 1092, 11, 427, 453, 479, 16, 432, 484, 692, 978, 1108, 694, 980, 1110]


In [13]:
%lprun -f game.observation_tensor game.observation_tensor(0)

In [14]:
%lprun -f game.step game.step(legal_actions[0])

In [15]:
%lprun -f agent.choose_best_action agent.choose_best_action(legal_actions)

In [16]:
%lprun -f model.evaluate model.evaluate(env, agents, num_episodes = 1)


100%|██████████| 1/1 [00:00<00:00,  2.96it/s]


In [17]:
from Backgammon import *
game = Backgammon(verbose=False, max_turns = 200)


In [18]:
%lprun -f game.get_legal_actions game.get_legal_actions()

In [19]:
%lprun -f game.legal_checker_moves_outer game.legal_checker_moves_outer(0, game.dice)

In [20]:
%lprun -f game.legal_checker_moves game.legal_checker_moves(game.cur_player, game.dice)

In [21]:
a = game.legal_checker_moves(game.cur_player, game.dice)[0]
a

Move(player=1, pos=23, num=4, hit=False)

In [22]:
%lprun -f game.apply_move game.apply_move(a)

In [24]:
%lprun -f game.update_board_pos game.update_board_pos(15, 1, 1)